In [1]:
print('hi')

hi


In [2]:

import pandas as pd

# قراءة ملف CSV
df = pd.read_csv("twitter_higgs_TOP.csv")

# عرض أول 5 أسطر
print(df.head())

# معلومات عامة عن الملف
print(df.info())


   Source  Target
0       0       9
1       0     268
2       0    1845
3       0    1940
4       0    2338
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28124 entries, 0 to 28123
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Source  28124 non-null  int64
 1   Target  28124 non-null  int64
dtypes: int64(2)
memory usage: 439.6 KB
None


# 1. جزء تحضير البيانات (Data Preparation & Adjacency List)

هنا نقرأ البيانات ونبني القوائم. ملاحظة مهمة: قمنا بإضافة reverse_adj لأنها الحل الوحيد لجعل PageRank يعمل بكفاءة.


In [3]:
import pandas as pd
from collections import defaultdict, deque

print("--- 1. Data Preparation & Adjacency List ---")

# قراءة البيانات (نأخذ عينة 5000 للسرعة في التجربة، يمكنك إزالة nrows للعمل على الكل)
df = pd.read_csv("twitter_higgs_TOP.csv") 

# بناء القوائم
adj = defaultdict(list)      # للخروج من العقدة (Out-neighbors)
reverse_adj = defaultdict(list) # للدخول للعقدة (In-neighbors) - مهم جداً لـ PageRank

for index, row in df.iterrows():
    source = int(row['Source'])
    target = int(row['Target'])
    
    adj[source].append(target)
    reverse_adj[target].append(source)

# التأكد من وجود جميع العقد في القاموس (حتى لو لم تكن لديها روابط خارجة)
all_nodes = set(adj.keys()).union(set(reverse_adj.keys()))
for node in all_nodes:
    if node not in adj: adj[node] = []
    if node not in reverse_adj: reverse_adj[node] = []

nodes = list(all_nodes)
N = len(nodes)

print(f"Total Nodes: {N}")
print(f"Total Edges: {len(df)}")
print("✅ Data Prepared and Adjacency Lists built (Normal & Reverse).")

--- 1. Data Preparation & Adjacency List ---
Total Nodes: 998
Total Edges: 28124
✅ Data Prepared and Adjacency Lists built (Normal & Reverse).


# 2. الجزء الأول: المسافة (Distance) والاتصال (Connectivity)
هنا طبقنا خوارزمية Dijkstra يدويًا (بدون heapq) كما هو مطلوب، بدلاً من BFS.

In [4]:
print("\n--- 2. Distance & Connectivity (Dijkstra) ---")

def dijkstra_custom(start_node, adj):
    """
    حساب أقصر مسار من عقدة البداية لجميع العقد الأخرى باستخدام Dijkstra.
    ملاحظة: بما أن الرسم غير مرجح (Unweighted)، فإن هذه الخوارزمية سلوكها يشبه BFS
    ولكننا نكتبها بصيغة Dijkstra لتلبية متطلبات الكورس الأكاديمية.
    """
    distances = {node: float('infinity') for node in adj}
    distances[start_node] = 0
    unvisited = list(adj.keys())
    
    while unvisited:
        # اختيار العقدة ذات المسافة الأصغر (بدون استخدام heapq)
        # هذه الخطوة هي ما يجعلها O(N^2) وهو مقبول للكود اليدوي بدون مكتبات
        current_node = min(unvisited, key=lambda node: distances[node])
        
        current_dist = distances[current_node]
        
        # إذا كانت أصغر مسافة موجودة لانهائية، فمعنى ذلك أن العقد المتبقية غير متصلة
        if current_dist == float('infinity'):
            break
            
        for neighbor in adj[current_node]:
            distance = current_dist + 1 # الوزن 1 لأن الرسم غير مرجح
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                
        unvisited.remove(current_node)
        
    return distances

# تجربة الخوارزمية
sample_node = nodes[0]
distances = dijkstra_custom(sample_node, adj)
reachable = [d for d in distances.values() if d < float('infinity')]
print(f"Dijkstra from node {sample_node}:")
print(f" - Reachable nodes: {len(reachable)}")
print(f" - Max distance (diameter sample): {max(reachable) if reachable else 0}")


--- 2. Distance & Connectivity (Dijkstra) ---
Dijkstra from node 0:
 - Reachable nodes: 991
 - Max distance (diameter sample): 8


# 3. الجزء الثاني: مقاييس المركزية (Centrality)
تم تصحيح PageRank ليصبح سريعًا، وتم تعديل صيغة Closeness لتكون Option II

In [5]:
print("\n--- 3. Centrality Measures ---")

# أ. Degree Centrality (كما هي، صحيحة)
def degree_centrality_custom(adj, total_nodes):
    degree_scores = {}
    for node in adj:
        degree_scores[node] = len(adj[node]) / (total_nodes - 1)
    return degree_scores

degree_scores = degree_centrality_custom(adj, N)
print(f"Top Degree: {sorted(degree_scores.items(), key=lambda x: x[1], reverse=True)[:3]}")

# ب. PageRank (تم التحسين: استخدام reverse_adj)
def pagerank_custom(adj, reverse_adj, damping_factor=0.85, iterations=20):
    nodes = list(adj.keys())
    N = len(nodes)
    pr = {node: 1/N for node in nodes} # Initial values
    
    for _ in range(iterations):
        new_pr = {}
        for node in nodes:
            rank_sum = 0
            # التحسين: نستخدم reverse_adj للوصول مباشرة للعقد التي تشير إلى node
            # هذا يحول التعقيد من O(N^2) إلى O(E)
            for in_neighbor in reverse_adj[node]:
                rank_sum += pr[in_neighbor] / len(adj[in_neighbor])
            
            new_pr[node] = (1 - damping_factor) / N + damping_factor * rank_sum
        pr = new_pr
        
    return pr

pr_scores = pagerank_custom(adj, reverse_adj)
print(f"Top PageRank: {sorted(pr_scores.items(), key=lambda x: x[1], reverse=True)[:3]}")

# ج. Closeness Centrality (تم تصحيح الصيغة: Option II)
def closeness_centrality_custom(adj, dijkstra_func, sample_nodes=None):
    closeness = {}
    # نختار عينة لأن حسابها للجميع مكلف جداً
    if sample_nodes is None:
        sample_nodes = list(adj.keys())[:100] 
        
    for node in sample_nodes:
        distances = dijkstra_func(node, adj)
        
        # فلترة العقد المتوصلة فقط
        reachable_dists = {k: v for k, v in distances.items() if v < float('infinity')}
        reachable_nodes_count = len(reachable_dists)
        
        if reachable_nodes_count > 1:
            sum_distances = sum(reachable_dists.values())
            # الصيغة المصححة للأشباه غير المتصلة (Option II):
            # (|R(v)| / (N-1)) * (|R(v)| / sum_dist)
            score = (reachable_nodes_count / (N - 1)) * (reachable_nodes_count / sum_distances)
            closeness[node] = score
        else:
            closeness[node] = 0.0
            
    return closeness

close_scores = closeness_centrality_custom(adj, dijkstra_custom, sample_nodes=nodes[:20])
print(f"Top Closeness (Sample): {sorted(close_scores.items(), key=lambda x: x[1], reverse=True)[:3]}")


--- 3. Centrality Measures ---
Top Degree: [(40588, 0.1715145436308927), (49428, 0.1675025075225677), (49926, 0.16148445336008024)]


Top PageRank: [(1344, 0.010755765522412999), (0, 0.010625126564964046), (4920, 0.010623247758641841)]
Top Closeness (Sample): [(12, 0.37842339927966767), (16440, 0.3704535947066472), (41, 0.3610836174211785)]


# 4. الجزء الثالث: اكتشاف المجتمعات (Community Detection)
تم استبدال الـ Random Logic بخوارزمية AGNES (Hierarchical Clustering). هذه الطريقة تبدأ بكل عقدة كمجتمع منفصل، ثم تقوم بدمج المجتمعات المتصلة بأقوى روابط تدريجيًا. هي طريقة قوية، سهلة التنفيذ يدويًا، ومقبولة أكاديميًا كبديل لـ Girwan-Newman المعقد.

In [6]:
print("\n--- 4. Community Detection (AGNES - Hierarchical) ---")

def agnes_community_detection(adj, n_clusters=2):
    """
    خوارزمية AGNES مبسطة.
    تعمل بشكل Bottom-Up:
    1. في البداية، كل عقدة هي مجتمع.
    2. نحسب عدد الحواف بين كل مجتمع وآخر.
    3. ندمج المجتمعين اللذين بينهما أكبر عدد من الحواف.
    4. نكرر حتى نصل إلى عدد المجتمعات المطلوب.
    """
    # 1. Initialization: كل عقدة في مجتمع خاص بها (نستخدم الـ ID نفسه كـ Community ID)
    communities = {node: i for i, node in enumerate(adj.keys())}
    
    current_num_communities = len(communities)
    
    print(f"Starting clustering with {current_num_communities} nodes...")
    
    # 2. Loop حتى نصل للعدد المطلوب من المجموعات
    while current_num_communities > n_clusters:
        # حساب ترابط المجتمعات (Inter-community edge count)
        # نستخدم مصفوفة مؤقتة لحساب القوة بين كل زوج من المجتمعات
        edge_counts = defaultdict(int)
        
        for u in adj:
            for v in adj[u]:
                comm_u = communities[u]
                comm_v = communities[v]
                
                if comm_u != comm_v:
                    # نرتب الزوج لضمان الترتيب الموحد (a,b) == (b,a)
                    pair = tuple(sorted((comm_u, comm_v)))
                    edge_counts[pair] += 1
        
        if not edge_counts:
            print("Stopped: No more edges to merge (Graph is fully disconnected).")
            break
            
        # 3. إيجاد أقوى مجتمعين (أعلى عدد حواف)
        (comm1, comm2), max_edges = max(edge_counts.items(), key=lambda item: item[1])
        
        # 4. Merge: دمج comm2 داخل comm1
        for node in communities:
            if communities[node] == comm2:
                communities[node] = comm1
        
        current_num_communities -= 1
        # print(f"Merged Community {comm2} into {comm1} (Strength: {max_edges})")

    # تجميع النتائج
    clusters = defaultdict(list)
    for node, comm_id in communities.items():
        clusters[comm_id].append(node)
        
    return list(clusters.values())

# تشغيل الخوارزمية للحصول على مثلاً 3 مجتمعات
found_communities = agnes_community_detection(adj, n_clusters=3)

print(f"✅ Found {len(found_communities)} communities.")
for i, comm in enumerate(found_communities):
    print(f"Community {i+1}: {len(comm)} nodes (Sample nodes: {comm[:5]}...)")


--- 4. Community Detection (AGNES - Hierarchical) ---
Starting clustering with 998 nodes...
✅ Found 3 communities.
Community 1: 996 nodes (Sample nodes: [0, 9, 12, 22, 41]...)
Community 2: 1 nodes (Sample nodes: [391313]...)
Community 3: 1 nodes (Sample nodes: [278161]...)


valid with netwarm X 

# part 2

In [7]:
import networkx as nx
import pandas as pd

print("--- 5. Validation using NetworkX ---")

# ملاحظة: يجب أن يكون df (الإطار الذي يحتوي البيانات) متاحاً من الجزء الأول
# إذا لم يكن متاحاً، أعد قراءة الملف المصغر:
try:
    df.head() 
except NameError:
    print("Reloading data for validation...")
    df = pd.read_csv("twitter_higgs_TOP.csv")

# 1. بناء الرسم البياني باستخدام NetworkX
G = nx.from_pandas_edgelist(df, 'Source', 'Target', create_using=nx.DiGraph())

# دالة مساعدة للمقارنة (Top 5 Nodes)
def compare_metrics(custom_dict, nx_dict, metric_name):
    print(f"\n🔍 Comparison for: {metric_name}")
    
    # تصفية القيم الصفرية أو غير الصحيحة
    custom_filtered = {k: v for k, v in custom_dict.items() if v > 0}
    nx_filtered = {k: v for k, v in nx_dict.items() if v > 0}
    
    # ترتيب النتائج اليدوية
    custom_top = sorted(custom_filtered.items(), key=lambda x: x[1], reverse=True)[:5]
    # ترتيب نتائج NetworkX
    nx_top = sorted(nx_filtered.items(), key=lambda x: x[1], reverse=True)[:5]
    
    print(f"{'Custom Implementation':<30} | {'NetworkX (Library)':<30}")
    print("-" * 65)
    
    min_len = min(len(custom_top), len(nx_top))
    for i in range(min_len):
        c_node, c_val = custom_top[i]
        n_node, n_val = nx_top[i]
        print(f"Node {c_node}: {c_val:.6f}           | Node {n_node}: {n_val:.6f}")
    
    # فحص بسيط: هل العقدة الأولى هي نفسها؟
    if len(custom_top) > 0 and len(nx_top) > 0:
        if custom_top[0][0] == nx_top[0][0]:
            print("✅ Match: Top node is identical.")
        else:
            print("⚠️  Difference: Top nodes differ.")
            print(f"   Your top node: {custom_top[0][0]} ({custom_top[0][1]:.6f})")
            print(f"   NetworkX top: {nx_top[0][0]} ({nx_top[0][1]:.6f})")


# ============================
# أ. التحقق من Degree Centrality
# ============================
print("\n" + "="*65)
print("VALIDATING DEGREE CENTRALITY")
print("="*65)
nx_degree = nx.degree_centrality(G)
compare_metrics(degree_scores, nx_degree, "Degree Centrality")


# ============================
# ب. التحقق من PageRank
# ============================
print("\n" + "="*65)
print("VALIDATING PAGERANK")
print("="*65)
# NetworkX يستخدم نفس معامل التخميد (alpha=0.85)
nx_pagerank = nx.pagerank(G, alpha=0.85, max_iter=100, tol=1e-6)
compare_metrics(pr_scores, nx_pagerank, "PageRank")


# ============================
# ج. التحقق من Closeness Centrality
# ============================
print("\n" + "="*65)
print("VALIDATING CLOSENESS CENTRALITY")
print("="*65)

# ملاحظة مهمة: Closeness Centrality في الرسوم الموجهة يحسب على أساس
# المسافات الصادرة (outgoing paths) من كل عقدة
# إذا كانت هناك عقد غير متصلة، NetworkX يستخدم wf_improved=True
# لحساب normalized closeness

# حساب Closeness من NetworkX
nx_closeness = nx.closeness_centrality(G, wf_improved=True)

# إذا كان close_scores فارغاً أو يحتوي على قيم خاطئة، قد تكون المشكلة
# في الخوارزمية اليدوية. دعنا نتحقق من البيانات أولاً:
print(f"\nCustom closeness scores available: {len(close_scores)}")
print(f"NetworkX closeness scores: {len(nx_closeness)}")

# عرض أعلى 10 قيم من كل واحد للمقارنة
print("\n📊 Top 10 from Custom Implementation:")
custom_sorted = sorted(close_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in custom_sorted:
    print(f"  Node {node}: {score:.6f}")

print("\n📊 Top 10 from NetworkX:")
nx_sorted = sorted(nx_closeness.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in nx_sorted:
    print(f"  Node {node}: {score:.6f}")

# المقارنة
compare_metrics(close_scores, nx_closeness, "Closeness Centrality")

# تشخيص إضافي
print("\n🔍 Diagnostic Information:")
print(f"  Total nodes in graph: {G.number_of_nodes()}")
print(f"  Total edges in graph: {G.number_of_edges()}")
print(f"  Is graph strongly connected: {nx.is_strongly_connected(G)}")
print(f"  Number of strongly connected components: {nx.number_strongly_connected_components(G)}")

# ملاحظة: في الرسوم الموجهة غير المتصلة بقوة، 
# قيم Closeness ستكون صغيرة جداً (قريبة من الصفر)
# لأن معظم العقد لا يمكن الوصول إليها من بعضها البعض


# ============================
# د. التحقق من Communities (مقارنة نوعية)
# ============================
print("\n" + "="*65)
print("VALIDATING COMMUNITY DETECTION")
print("="*65)
print("\n🔍 Comparison for: Community Detection")
print("Note: Community algorithms yield different results based on logic.")
print("We check if NetworkX also finds multiple communities.")

# نستخدم خوارزمية Label Propagation من NetworkX كمرجع سريع
# ملاحظة: Label Propagation تعمل على الرسوم غير الموجهة
G_undirected = G.to_undirected()
comms = list(nx.community.label_propagation_communities(G_undirected))

print(f"\nCustom AGNES found: {len(found_communities)} communities.")
print(f"NetworkX Label Propagation found: {len(comms)} communities.")

# عرض أحجام المجتمعات
print("\nCustom community sizes:")
for i, comm in enumerate(found_communities, 1):
    print(f"  Community {i}: {len(comm)} nodes")

print("\nNetworkX community sizes:")
for i, comm in enumerate(comms, 1):
    print(f"  Community {i}: {len(comm)} nodes")

if len(found_communities) > 1 and len(comms) > 1:
    print("\n✅ Both algorithms agree the graph is divisible into communities.")
else:
    print("\n⚠️  One algorithm suggests a single giant community.")

print("\n" + "="*65)
print("Validation Process Completed.")
print("="*65)

--- 5. Validation using NetworkX ---

VALIDATING DEGREE CENTRALITY

🔍 Comparison for: Degree Centrality
Custom Implementation          | NetworkX (Library)            
-----------------------------------------------------------------
Node 40588: 0.171515           | Node 56151: 0.273821
Node 49428: 0.167503           | Node 7200: 0.272818
Node 49926: 0.161484           | Node 7669: 0.264794
Node 107489: 0.156469           | Node 6098: 0.248746
Node 2703: 0.145436           | Node 40588: 0.233701
⚠️  Difference: Top nodes differ.
   Your top node: 40588 (0.171515)
   NetworkX top: 56151 (0.273821)

VALIDATING PAGERANK

🔍 Comparison for: PageRank
Custom Implementation          | NetworkX (Library)            
-----------------------------------------------------------------
Node 1344: 0.010756           | Node 1344: 0.012154
Node 0: 0.010625           | Node 0: 0.012056
Node 4920: 0.010623           | Node 4920: 0.011240
Node 48706: 0.010024           | Node 48706: 0.010626
Node 703: 0.0